# *ETL usando API 

ETL é um conjunto de instruções de computador que trata da extração de dados de seu sistema de origem, transformação de dados para atender a várias necessidades de inteligência de negócios e carregamento de dados em alguns sistemas de destino. 

API significa interface de programação de aplicativo. Em essência, uma API atua como uma camada de comunicação, ou como o nome diz, uma interface, que permite que diferentes sistemas conversem entre si sem ter que entender exatamente o que cada um faz. Uma das coleções mais legais de APIs publicamente disponíveis é a fornecida pela NASA. O objetivo do site é tornar os dados da NASA, incluindo imagens, eminentemente acessíveis aos desenvolvedores de aplicativos. 

MySQL é um sistema gerenciador de banco de dados relacional de código aberto usado na maioria das aplicações gratuitas para gerir suas bases de dados. O serviço utiliza a linguagem SQL (Structure Query Language – Linguagem de Consulta Estruturada), que é a linguagem mais popular para inserir, acessar e gerenciar o conteúdo armazenado num banco de dados.

## Objetivo

Criar um pipeline ETL onde retiramos dados de um API da NASA sobre ejeção de massa coronal, transformamos e limpamos os dados em uma nova tabela e, por último, carregamos os dados no banco de dados MySQL.

In [16]:
# importando as bibliotecas
import numpy as np
import pandas as pd 
from datetime import datetime
import json
from pandas import json_normalize
import requests
import re

import mysql.connector
from mysql.connector import Error, connect

## 1. Conectando-se e Extraindo os Dados

A primeira etapa é conectar-se aos dados. 

In [2]:
# endereço do site
url = "https://api.nasa.gov/DONKI/CME?"

Embora muitas APIs públicas sejam gratuitas e completamente públicas, um número ainda maior de APIs está disponível por trás de alguma forma de autenticação. O nível mais comum de autenticação é a chave API. Essas chaves são usadas para identificá-lo como um usuário ou cliente da API e para rastrear seu uso da API. As chaves de API são normalmente enviadas como um cabeçalho de solicitação ou como um parâmetro de consulta. Para nossos fins, podemos usar a chave API DEMO_KEY que a NASA fornece por padrão. 

In [3]:
param = {"api_key":"DEMO_KEY",
        "startDate":"2020-01-01",
        "endDate":"2021-06-01"}

A biblioteca 'request' busca e pega os dados da URL.

In [4]:
# chamamos a API
response = requests.get(url,params=param)

Os códigos de status são uma das informações mais importantes a serem procuradas em qualquer resposta da API. Eles informam se sua solicitação foi bem-sucedida, se faltam dados, se faltam credenciais e assim por diante. Para o requerimento ser sucesso precisamos de um códico de acesso 200.

Por questão de confirmação da busca, podemos exibir o primeiro elemento do conteúdo de dados no formato JSON.

In [5]:
# imprime o status code
print(response.status_code)

print(response.json()[0])

200
{'activityID': '2020-01-05T16:45:00-CME-001', 'catalog': 'M2M_CATALOG', 'startTime': '2020-01-05T16:45Z', 'sourceLocation': '', 'activeRegionNum': None, 'link': 'https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CME/15256/-1', 'note': 'The source is likely a field line opening seen at 09:45Z on the western limb in EUVIA 195.', 'instruments': [{'displayName': 'STEREO A: SECCHI/COR2'}, {'displayName': 'SOHO: LASCO/C2'}, {'displayName': 'SOHO: LASCO/C3'}], 'cmeAnalyses': [{'time21_5': '2020-01-06T16:13Z', 'latitude': -2.0, 'longitude': 9.0, 'halfAngle': 19.0, 'speed': 227.0, 'type': 'S', 'isMostAccurate': True, 'note': '', 'levelOfData': 0, 'link': 'https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CMEAnalysis/15257/-1', 'enlilList': [{'modelCompletionTime': '2020-01-06T21:23Z', 'au': 2.0, 'estimatedShockArrivalTime': None, 'estimatedDuration': None, 'rmin_re': None, 'kp_18': None, 'kp_90': None, 'kp_135': None, 'kp_180': None, 'isEarthGB': False, 'link': 'https://kauai.ccmc.gsfc.nasa.gov/DONKI/v

Temos agora que decodificar os dados do formato JSON para CSV.

In [8]:
# decodifica os dados JSON para Python
data = json.loads(response.text)
# Normalizar dados JSON semiestruturados em uma tabela plana 
df = json_normalize(data)
# configura para formato csv
df.to_csv('upload_file.csv', index = False)

## 2. Processando e Transformando os Dados

Deposse dos dados, podemos fazer as alterações para selecionar apenas algumas informações desejadas.

In [9]:
# carrega os dados em um DataFrame
upload = pd.read_csv("upload_file.csv")
# exibe o número de linhas e colunas
print(upload.shape)
# imprime as primeiras 5 linhas
upload.head()

(277, 10)


,activityID,catalog,startTime,sourceLocation,activeRegionNum,link,note,instruments,cmeAnalyses,linkedEvents
0,2020-01-05T16:45:00-CME-001,M2M_CATALOG,2020-01-05T16:45Z,NaN,NaN,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...,The source is likely a field line opening seen...,"[{'displayName': 'STEREO A: SECCHI/COR2'}, {'d...","[{'time21_5': '2020-01-06T16:13Z', 'latitude':...",NaN
1,2020-01-14T11:09:00-CME-001,M2M_CATALOG,2020-01-14T11:09Z,S05W20,NaN,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...,"It is a super small, narrow and weak CME. The ...",[{'displayName': 'STEREO A: SECCHI/COR2'}],"[{'time21_5': '2020-01-15T03:44Z', 'latitude':...",NaN
2,2020-01-18T20:12:00-CME-001,M2M_CATALOG,2020-01-18T20:12Z,NaN,NaN,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...,The source is unclear and the longitude is an ...,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...","[{'time21_5': '2020-01-19T23:11Z', 'latitude':...",NaN
3,2020-01-22T09:09:00-CME-001,M2M_CATALOG,2020-01-22T09:09Z,NaN,NaN,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...,There is a data gap in Stereo Ahead EUVI 195 f...,[{'displayName': 'STEREO A: SECCHI/COR2'}],"[{'time21_5': '2020-01-22T22:58Z', 'latitude':...",NaN
4,2020-01-25T18:54:00-CME-001,M2M_CATALOG,2020-01-25T18:54Z,N04E20,12757.0,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...,The source of this CME seems to be to be relat...,[{'displayName': 'STEREO A: SECCHI/COR2'}],"[{'time21_5': '2020-01-26T04:27Z', 'latitude':...",NaN


Vamos selecionar apenas as informações referentes a dia, mês, ano e hora da ocorrência de ejeção de massa coronal, além de informações da latitude, longitude, ângulo e velocidade da ejeção.

In [10]:
# cria um novo DataFrame
df = pd.DataFrame()

# pega a data
df["data"] = upload.startTime.str[0:10]
# converte para o formato pandas datatime
df["data"] = pd.to_datetime(df.data)

df["ano"] = df['data'].dt.year # adiciona o ano
df["mes"] = df["data"].dt.month # adidiona o M~es
df['dia_semana'] = df['data'].dt.weekday # adiciona o dia da semana
df["dia"] = df["data"].dt.day # adiciona o dia do mês
df["hora"] = upload.startTime.str[12:16]  

# apaga a coluna data
df.drop("data", axis=1, inplace=True)

In [11]:
# cria novas colunas com valores nulos 
df["latitude"] = np.nan
df["longitude"] = np.nan
df["halfAngle"] = np.nan
df["velocidade"] = np.nan 

for name in upload["cmeAnalyses"].values:
    # busca pelas expressões dentro de '[}]' na coluna 'cmeAnalyses'
    if re.search('[{]', str(name)):
        # localiza o índice da linha
        idx = upload[upload["cmeAnalyses"] == name].index.values
        # divide a string e pega o valor da latitude
        latitude = str(name).split(sep=":")[3].split(",")[0]
        # adiciona na coluna latitude
        df.loc[idx, "latitude"] = str(latitude)
        
        #longitude
        longitude = str(name).split(sep=":")[4].split(",")[0]
        df.loc[idx, "longitude"] = str(longitude)
        
        # ângulo
        halfAngle = str(name).split(sep=":")[5].split(",")[0]
        df.loc[idx, "halfAngle"] = str(halfAngle)
        
        # velocidade
        velocidade = str(name).split(sep=":")[6].split(",")[0]
        df.loc[idx, "velocidade"] = str(velocidade)

Pronto! Temos uma nova tabela com os dados limpos. 

In [12]:
#exibe o DataFrame limpo
df.head()

,ano,mes,dia_semana,dia,hora,latitude,longitude,halfAngle,velocidade
0,2020,1,6,5,6:45,-2.0,9.0,19.0,227.0
1,2020,1,1,14,1:09,-5.0,12.0,6.0,205.0
2,2020,1,5,18,0:12,5.0,-115.0,17.0,193.0
3,2020,1,2,22,9:09,2.0,None,18.0,248.0
4,2020,1,5,25,8:54,2.0,-17.0,10.0,362.0


## 3. Carregando no MySQL 

A última etapa é salvar a tabela no banco de dados.

In [19]:
# conecta ao mysql
cnx = connect(user='root', passwd='bankai', host='localhost')
# cria um cursor para navegar no mysql
cursor = cnx.cursor()

# criando um novo banco de dados chamado 'coronal_mass_ejection'
cursor.execute("CREATE DATABASE coronal_mass_ejection") 
# acessa o banco de dados 
cursor.execute('USE coronal_mass_ejection')


In [26]:
# cria uma tabela chamada 'coronal' para inserir os dados
cursor.execute("CREATE TABLE coronal (ano INT(64), mes INT(64), dia_semana INT(64), \
                                    dia INT(64), hora TIME, latitude INT(64), \
                                    longitude INT(64), halfAngle INT(64), velocidade INT(64))")

Para facilitar a inserção dos dados, vamos criar uma função que realize essa tarefa.

In [24]:
# função para inserir os dados na tabela
def insert_data(coronal):
    query = "INSERT INTO coronal (ano, mes, dia_semana, dia, hora, latitude, longitude, halfAngle, velocidade)\
             VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

    try:
        # insere os dados na tabela
        cursor.executemany(query, coronal)

    except Error as e:
        # caso dê erro, imprime mensagem com o erro
        print('Error:', e)

In [28]:
# remove os dados nulos, caso existam
df_null = df.replace(np.nan, 'NULL', regex=True)

# configura de uma tabela DataFrame para uma lista 
coronal_list = df_null.values.tolist()

# insere a lista na tabela do banco de dados com a ajuda da função
insert_data(coronal_list)

# commitando os dados adicionados
cnx.commit()

# imprime quantas linhas de dados foram inseridas no banco de dados
print(cursor.rowcount, "linhas foram inseridas.")

277 linhas foram inseridas.


In [29]:
# fecha a conexão com o mysql
cnx.close()

Pronto! Temos nosso pipeline finalizado.

## 4. Conclusão

Criamos um modelo de pipeline ETL que extraí dados de uma API da NASA, transforma e limpa os dados e os salva em um banco de dados MySQL.